<a href="https://colab.research.google.com/github/vanimalhotra22/Capstone-Project/blob/main/final_project_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# -*- coding: utf-8 -*-
"""
# Dynamic Pricing for Urban Parking Lots - Capstone Project

This Google Colab notebook implements a dynamic pricing engine for urban parking lots,
simulating real-time data streams and applying various pricing models.

**Core Requirements Addressed:**
- **Intelligent, Data-Driven Pricing:** Implements three models (Baseline, Demand-Based, Competitive).
- **14 Parking Spaces:** Data simulated for 14 distinct parking spaces.
- **Real-Time Data Streams (Pathway):** Utilizes Pathway for data ingestion, real-time processing, and continuous prediction.
- **Models from Scratch:** Pricing logic implemented using only NumPy and Pandas.
- **Price Update Logic:** Prices updated based on historical occupancy, queue length, nearby traffic, special events, vehicle type, and competitor prices (for Model 3).
- **Base Price of $10:** All pricing models start from a base price of $10.
- **Smooth and Explainable Price Variation:** Pricing functions are designed to be smooth.
- **Bokeh Visualizations:** Real-time line plots for pricing and comparison.

**Execution Environment:** Google Colab.
"""

# --- CELL 1: Setup and Imports ---
# This cell installs necessary libraries and imports all modules required for the project.

print("--- CELL 1: Setting up environment and importing libraries ---")

# Install Pathway - This command is specific to Google Colab.
# If running locally, ensure Pathway is installed: pip install pathway
!pip install pathway bokeh numpy pandas

import pathway as pw
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import random
from math import radians, sin, cos, sqrt, atan2

# Bokeh for real-time visualization
from bokeh.plotting import figure, show, curdoc
from bokeh.models import ColumnDataSource, DatetimeTickFormatter, NumeralTickFormatter, CustomJS
from bokeh.layouts import column, row
from bokeh.palettes import Category10, Category20

# Import output_file and save for HTML export
from bokeh.io import output_file, save

print("Environment setup complete. Libraries imported.")

# --- CELL 2: Data Simulation (dataset.csv generation) ---
# This cell generates a synthetic dataset that mimics the structure described
# in the problem statement. This CSV will then be streamed by Pathway.

print("\n--- CELL 2: Generating synthetic dataset.csv ---")

# Configuration for data generation
NUM_PARKING_SPACES = 14
DAYS_OF_DATA = 73
TIME_POINTS_PER_DAY = 18 # From 8:00 AM to 4:30 PM with 30 min difference
BASE_PRICE = 10.0

# Fixed properties for each parking space
parking_space_data = []
for i in range(NUM_PARKING_SPACES):
    parking_space_data.append({
        'parking_space_id': i,
        'latitude': round(random.uniform(34.0, 34.1), 5), # Example latitude range
        'longitude': round(random.uniform(-118.3, -118.2), 5), # Example longitude range
        'capacity': random.randint(50, 200) # Capacity between 50 and 200 vehicles
    })
parking_spaces_df = pd.DataFrame(parking_space_data)

# Generate time series data
all_records = []
start_date = datetime(2025, 1, 1) # Start date for simulation

for day in range(DAYS_OF_DATA):
    current_date = start_date + timedelta(days=day)
    for time_point in range(TIME_POINTS_PER_DAY):
        current_time = (datetime(1, 1, 1, 8, 0) + timedelta(minutes=time_point * 30)).time()
        timestamp = datetime.combine(current_date.date(), current_time)

        for _, space_info in parking_spaces_df.iterrows():
            space_id = space_info['parking_space_id']
            capacity = space_info['capacity']
            latitude = space_info['latitude']
            longitude = space_info['longitude']

            # Simulate dynamic features
            # Occupancy: Peaks during mid-day
            hour = current_time.hour + current_time.minute / 60
            occupancy_base = capacity * (0.3 + 0.6 * sin(np.pi * (hour - 8) / (16.5 - 8))) # Peak around noon-afternoon
            occupancy = int(max(0, min(capacity, occupancy_base + random.randint(-5, 5))))

            # Queue length: Higher with high occupancy
            queue_length = max(0, int((occupancy / capacity) * random.randint(0, 10) - 2))
            if occupancy >= capacity * 0.9: # More queue if nearly full
                queue_length += random.randint(0, 5)

            # Traffic congestion: Peaks during rush hours (morning/evening)
            traffic_level = 0
            if 8 <= hour <= 9.5 or 15.5 <= hour <= 16.5: # Morning/Evening rush
                traffic_level = random.uniform(0.6, 1.0)
            else:
                traffic_level = random.uniform(0.2, 0.7)

            # Special day indicator (e.g., weekend or holiday)
            is_special_day = 1 if current_date.weekday() >= 5 or random.random() < 0.05 else 0 # Weekends or 5% chance of holiday

            # Vehicle type mix (simplified as dominant type for the record)
            vehicle_type = random.choice(['car', 'bike', 'truck'])

            # Simulate competitor prices (for Model 3) - simplified for now
            # In a real scenario, this would come from external feeds
            competitor_price = round(BASE_PRICE * random.uniform(0.8, 1.2), 2)

            all_records.append({
                'timestamp': timestamp,
                'parking_space_id': space_id,
                'latitude': latitude,
                'longitude': longitude,
                'capacity': capacity,
                'occupancy': occupancy,
                'queue_length': queue_length,
                'traffic_level': traffic_level,
                'is_special_day': is_special_day,
                'vehicle_type': vehicle_type,
                'competitor_price': competitor_price,
                'base_price': BASE_PRICE # Store base price for reference
            })

df_dataset = pd.DataFrame(all_records)
df_dataset = df_dataset.sort_values(by=['timestamp', 'parking_space_id']).reset_index(drop=True)

# Save the dataset to a CSV file
DATASET_FILE = 'dataset.csv'
df_dataset.to_csv(DATASET_FILE, index=False)
print(f"Synthetic dataset '{DATASET_FILE}' generated with {len(df_dataset)} records.")
print("First 5 rows of the generated dataset:")
print(df_dataset.head())


# --- CELL 3: Pathway Data Ingestion & Base Data Stream ---
# This cell sets up Pathway to ingest the simulated data as a real-time stream.

print("\n--- CELL 3: Setting up Pathway data ingestion ---")

# Clear any previous Pathway context (removed pw.debug.reset() as it caused an AttributeError)
# pw.debug.reset() # This line was removed due to AttributeError

# Define the input stream from the CSV file
# We use `mode='csv'` and `autocommit_duration_ms=100` to simulate streaming
# with a small delay for real-time updates.
input_stream = pw.io.csv.read(
    DATASET_FILE,
    schema=pw.schema_from_csv(DATASET_FILE),
    autocommit_duration_ms=100, # Simulate real-time updates every 100ms
)

print(f"Pathway input stream configured from '{DATASET_FILE}'.")
print("Schema of the input stream:")
print(input_stream.schema)


# --- CELL 4: Pricing Model 1: Baseline Linear Model ---
# Implements a simple linear pricing model based on occupancy rate.

print("\n--- CELL 4: Implementing Pricing Model 1 (Baseline Linear) ---")

# Model 1: Baseline Linear Model
# Price_t+1 = Price_t + alpha * (Occupancy / Capacity)
# We'll use the current occupancy/capacity to adjust from the base price.
# For streaming, Price_t is effectively the base_price.

# Define the pricing function as a Pathway UDF
@pw.udf
def calculate_price_model1_udf(occupancy: int, capacity: int, base_price: float) -> float:
    alpha = 5.0 # Sensitivity parameter for price increase
    occupancy_rate = occupancy / capacity
    # Ensure price is always positive and within reasonable bounds
    new_price = base_price + alpha * occupancy_rate
    return round(max(0.1 * base_price, min(2.0 * base_price, new_price)), 2)

# Get existing column names from the input_stream schema
input_stream_columns = list(input_stream.schema.keys())

# Apply Model 1 to the input stream using .select and the UDF
prices_model1 = input_stream.select(
    *[getattr(pw.this, col) for col in input_stream_columns], # Dynamically select all existing columns
    price_model1=calculate_price_model1_udf(
        pw.this.occupancy,
        pw.this.capacity,
        pw.this.base_price
    )
)

print("Pricing Model 1 defined and applied to the stream using UDF.")
print("Example of Model 1 output schema:")
print(prices_model1.schema)


# --- CELL 5: Pricing Model 2: Demand-Based Price Function ---
# Implements a more advanced model using a custom demand function.

print("\n--- CELL 5: Implementing Pricing Model 2 (Demand-Based) ---")

# Model 2: Demand-Based Price Function
# Demand = alpha * (Occupancy / Capacity) + beta * Queue Length + gamma * Traffic + delta * IsSpecialDay + epsilon * VehicleTypeWeight
# Price = Base Price * (1 + lambda * NormalizedDemand)

# Assumptions for demand function weights:
# - Occupancy rate has a strong positive impact on demand.
# - Queue length indicates high demand.
# - Traffic level indicates high demand.
# - Special day increases demand.
# - Vehicle type weights: Cars might have higher demand weight than bikes/trucks.
VEHICLE_TYPE_WEIGHTS = {
    'car': 0.8,
    'bike': 0.2,
    'truck': 0.5
}

# Define the demand calculation as a Pathway UDF
@pw.udf
def calculate_demand_udf(occupancy: int, capacity: int, queue_length: int, traffic_level: float, is_special_day: int, vehicle_type: str) -> float:
    # VEHICLE_TYPE_WEIGHTS needs to be accessible within the UDF's scope or passed as an argument
    # For simplicity in UDF, we'll redefine it or assume it's globally available (less ideal for UDFs)
    # Best practice is to pass all necessary parameters.
    # For this demo, we'll put it inside the UDF for self-containment.
    VEHICLE_TYPE_WEIGHTS_UDF = {
        'car': 0.8,
        'bike': 0.2,
        'truck': 0.5
    }

    alpha = 0.6 # Weight for occupancy rate
    beta = 0.1  # Weight for queue length
    gamma = 0.2 # Weight for traffic level
    delta = 0.3 # Weight for special day
    epsilon = 0.1 # Weight for vehicle type

    occupancy_rate = occupancy / capacity
    queue_contribution = queue_length / 10.0 # Normalize queue length
    traffic_contribution = traffic_level
    special_day_contribution = is_special_day
    vehicle_type_contribution = VEHICLE_TYPE_WEIGHTS_UDF.get(vehicle_type, 0.0)

    demand = (alpha * occupancy_rate +
              beta * queue_contribution +
              gamma * traffic_contribution +
              delta * special_day_contribution +
              epsilon * vehicle_type_contribution)
    return demand

# Define the price calculation for Model 2 as a Pathway UDF
@pw.udf
def calculate_price_model2_udf(base_price: float, demand_value: float) -> float:
    lambda_factor = 0.5 # Sensitivity of price to demand
    normalized_demand = min(1.0, max(0.0, demand_value / 1.5)) # Scale demand to be between 0 and 1

    price_multiplier = 1 + lambda_factor * normalized_demand
    price_multiplier = max(0.5, min(2.0, price_multiplier)) # Ensure price_multiplier is within [0.5, 2.0]

    new_price = base_price * price_multiplier
    return round(new_price, 2)

# Get existing column names from prices_model1 schema
prices_model1_columns = list(prices_model1.schema.keys())

# First, calculate demand as an intermediate column
prices_with_demand = prices_model1.select(
    *[getattr(pw.this, col) for col in prices_model1_columns], # Dynamically select all existing columns
    demand_value=calculate_demand_udf(
        pw.this.occupancy,
        pw.this.capacity,
        pw.this.queue_length,
        pw.this.traffic_level,
        pw.this.is_special_day,
        pw.this.vehicle_type
    )
)

# Get existing column names from prices_with_demand schema
prices_with_demand_columns = list(prices_with_demand.schema.keys())

# Then, calculate price_model2 using the demand_value
prices_model2 = prices_with_demand.select(
    *[getattr(pw.this, col) for col in prices_with_demand_columns], # Dynamically select all existing columns
    price_model2=calculate_price_model2_udf(
        pw.this.base_price,
        pw.this.demand_value
    )
)

print("Pricing Model 2 defined and applied to the stream using UDFs.")
print("Example of Model 2 output schema:")
print(prices_model2.schema)


# --- CELL 6: Pricing Model 3: Competitive Pricing Model ---
# Adds location intelligence and factors in competitor prices.

print("\n--- CELL 6: Implementing Pricing Model 3 (Competitive) ---")

# Model 3: Competitive Pricing Model
# Factors in geographic proximity and competitor prices.

# Haversine formula to calculate distance between two lat/long points
# This function is used in the simulation loop, not directly as a Pathway UDF
# because it requires iterating over other records in a timestep, which is complex for UDFs.
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371 # Radius of Earth in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Define the pricing calculation for Model 3 as a Pathway UDF
@pw.udf
def calculate_price_model3_udf(price_model2: float, occupancy: int, capacity: int, base_price: float, competitor_price: float) -> float:
    current_price = price_model2
    occupancy_rate = occupancy / capacity
    avg_competitor_price = competitor_price # Using the pre-simulated competitor price

    # Competitive logic:
    if occupancy_rate > 0.95 and current_price > avg_competitor_price * 1.05: # 5% more expensive
        current_price *= 0.98 # Small reduction
    elif occupancy_rate < 0.8 and current_price < avg_competitor_price * 0.95: # 5% cheaper
        current_price *= 1.02 # Small increase

    # Ensure price bounds are maintained
    return round(max(0.1 * base_price, min(2.0 * base_price, current_price)), 2)

# Get existing column names from prices_model2 schema
prices_model2_columns = list(prices_model2.schema.keys())

# Apply Model 3 to the stream
prices_model3 = prices_model2.select(
    *[getattr(pw.this, col) for col in prices_model2_columns], # Dynamically select all existing columns
    price_model3=calculate_price_model3_udf(
        pw.this.price_model2,
        pw.this.occupancy,
        pw.this.capacity,
        pw.this.base_price,
        pw.this.competitor_price
    )
)

print("Pricing Model 3 defined and applied to the stream using UDF.")
print("Example of Model 3 output schema:")
print(prices_model3.schema)


# --- CELL 7: Bokeh Visualization Setup ---
# This cell sets up the Bokeh plots and ColumnDataSource for real-time updates.

print("\n--- CELL 7: Setting up Bokeh visualizations ---")

# Create a ColumnDataSource to hold the streaming data for Bokeh plots
# Initialize with empty data for all parking spaces and models
source_data = {
    # Removed general lists as they are not used for plotting individual lines
    # and were causing length mismatches with space-specific lists.
}
# For each parking space, we need separate lists for prices over time
for i in range(NUM_PARKING_SPACES):
    source_data[f'price_model1_space_{i}'] = []
    source_data[f'price_model2_space_{i}'] = []
    source_data[f'price_model3_space_{i}'] = []
    source_data[f'competitor_price_space_{i}'] = []
    source_data[f'occupancy_rate_space_{i}'] = []
    source_data[f'time_space_{i}'] = []


source = ColumnDataSource(data=source_data)

# Create Bokeh plots
# Plot 1: Real-time Pricing for each parking space (Model 3)
p1 = figure(
    x_axis_type="datetime",
    height=400,
    width=900,
    title="Real-time Dynamic Pricing (Model 3)",
    x_axis_label="Time",
    y_axis_label="Price ($)",
    tools="pan,wheel_zoom,box_zoom,reset,save",
    sizing_mode="scale_width"
)
p1.xaxis.formatter = DatetimeTickFormatter(
    hours="%H:%M",
    days="%m/%d %H:%M",
    months="%m/%d %H:%M",
    years="%Y-%m-%d %H:%M"
)
p1.yaxis.formatter = NumeralTickFormatter(format="$0.00")

# Add lines for each parking space's Model 3 price
# Using Category20 palette for distinct colors, as Category10 is too small for 14 spaces
colors = Category20[NUM_PARKING_SPACES]
for i in range(NUM_PARKING_SPACES):
    p1.line(
        x=f'time_space_{i}',
        y=f'price_model3_space_{i}',
        source=source,
        legend_label=f'Space {i} (Model 3)',
        color=colors[i % len(colors)],
        line_width=2,
        alpha=0.7
    )
p1.legend.location = "top_left"
p1.legend.click_policy = "hide" # Allow hiding lines by clicking legend

# Plot 2: Comparison of Models for a single parking space (e.g., Space 0)
p2 = figure(
    x_axis_type="datetime",
    height=350,
    width=900,
    title="Price Model Comparison for Parking Space 0",
    x_axis_label="Time",
    y_axis_label="Price ($)",
    tools="pan,wheel_zoom,box_zoom,reset,save",
    sizing_mode="scale_width"
)
p2.xaxis.formatter = DatetimeTickFormatter(
    hours="%H:%M",
    days="%m/%d %H:%M",
    months="%m/%d %H:%M",
    years="%Y-%m-%d %H:%M"
)
p2.yaxis.formatter = NumeralTickFormatter(format="$0.00")

p2.line(x='time_space_0', y='price_model1_space_0', source=source, legend_label='Model 1 (Baseline)', color='red', line_width=2)
p2.line(x='time_space_0', y='price_model2_space_0', source=source, legend_label='Model 2 (Demand-Based)', color='green', line_width=2)
p2.line(x='time_space_0', y='price_model3_space_0', source=source, legend_label='Model 3 (Competitive)', color='blue', line_width=2)
p2.line(x='time_space_0', y='competitor_price_space_0', source=source, legend_label='Competitor Price (Space 0)', color='orange', line_dash='dashed', line_width=2)
p2.legend.location = "top_left"
p2.legend.click_policy = "hide"

# Plot 3: Occupancy Rate for a single parking space (e.g., Space 0)
p3 = figure(
    x_axis_type="datetime",
    height=250,
    width=900,
    title="Occupancy Rate for Parking Space 0",
    x_axis_label="Time",
    y_axis_label="Occupancy Rate",
    tools="pan,wheel_zoom,box_zoom,reset,save",
    sizing_mode="scale_width"
)
p3.xaxis.formatter = DatetimeTickFormatter(
    hours="%H:%M",
    days="%m/%d %H:%M",
    months="%m/%d %H:%M",
    years="%Y-%m-%d %H:%M"
)
p3.yaxis.formatter = NumeralTickFormatter(format="0%")
p3.line(x='time_space_0', y='occupancy_rate_space_0', source=source, legend_label='Occupancy Rate', color='purple', line_width=2)
p3.legend.location = "top_left"


# Arrange plots in a layout
layout = column(p1, p2, p3)

print("Bokeh plots and data sources initialized.")


# --- CELL 8: Pathway to Bokeh Integration & Real-time Loop ---
# This cell connects the Pathway stream to Bokeh's ColumnDataSource
# and starts the real-time simulation.

print("\n--- CELL 8: Starting Pathway-Bokeh real-time simulation ---")

# Define the update function for Bokeh
def update_bokeh_data(new_data): # Removed bokeh_handle as it's not needed for HTML export
    # Convert Pandas Series (records) to a format suitable for Bokeh's ColumnDataSource
    if not new_data:
        return

    # Prepare new data for appending to ColumnDataSource.
    # Each list in new_patch will contain exactly one element, corresponding to the
    # data for the current timestamp for that specific parking space's line.
    new_patch = {}

    for record in new_data:
        space_id = int(record['parking_space_id']) # Ensure it's an int

        # For each space, prepare a single-element list for each relevant column
        new_patch[f'time_space_{space_id}'] = [record['timestamp']]
        new_patch[f'price_model1_space_{space_id}'] = [record['price_model1']]
        new_patch[f'price_model2_space_{space_id}'] = [record['price_model2']]
        new_patch[f'price_model3_space_{space_id}'] = [record['price_model3']]
        new_patch[f'competitor_price_space_{space_id}'] = [record['competitor_price']]
        new_patch[f'occupancy_rate_space_{space_id}'] = [record['occupancy'] / record['capacity']]

    # Update Bokeh ColumnDataSource
    # We use stream() for appending new data. All lists in new_patch now have length 1.
    source.stream(new_patch, rollover=1000) # Rollover to keep the plot from growing indefinitely
    # push_notebook(handle=bokeh_handle) # Removed as we are exporting to HTML now

# Define plain Python functions for pricing calculations, mirroring the UDF logic
# These will be used in the manual simulation loop to avoid Pathway expression serialization issues.
def _calculate_price_model1(occupancy: int, capacity: int, base_price: float) -> float:
    alpha = 5.0
    occupancy_rate = occupancy / capacity
    new_price = base_price + alpha * occupancy_rate
    return round(max(0.1 * base_price, min(2.0 * base_price, new_price)), 2)

def _calculate_demand(occupancy: int, capacity: int, queue_length: int, traffic_level: float, is_special_day: int, vehicle_type: str) -> float:
    VEHICLE_TYPE_WEIGHTS_LOCAL = {
        'car': 0.8,
        'bike': 0.2,
        'truck': 0.5
    }
    alpha = 0.6
    beta = 0.1
    gamma = 0.2
    delta = 0.3
    epsilon = 0.1

    occupancy_rate = occupancy / capacity
    queue_contribution = queue_length / 10.0
    traffic_contribution = traffic_level
    special_day_contribution = is_special_day
    vehicle_type_contribution = VEHICLE_TYPE_WEIGHTS_LOCAL.get(vehicle_type, 0.0)

    demand = (alpha * occupancy_rate +
              beta * queue_contribution +
              gamma * traffic_contribution +
              delta * special_day_contribution +
              epsilon * vehicle_type_contribution)
    return demand

def _calculate_price_model2(base_price: float, demand_value: float) -> float:
    lambda_factor = 0.5
    normalized_demand = min(1.0, max(0.0, demand_value / 1.5))
    price_multiplier = 1 + lambda_factor * normalized_demand
    price_multiplier = max(0.5, min(2.0, price_multiplier))
    new_price = base_price * price_multiplier
    return round(new_price, 2)

def _calculate_price_model3(price_model2: float, occupancy: int, capacity: int, base_price: float, competitor_price: float) -> float:
    current_price = price_model2
    occupancy_rate = occupancy / capacity
    avg_competitor_price = competitor_price

    if occupancy_rate > 0.95 and current_price > avg_competitor_price * 1.05:
        current_price *= 0.98
    elif occupancy_rate < 0.8 and current_price < avg_competitor_price * 0.95:
        current_price *= 1.02
    return round(max(0.1 * base_price, min(2.0 * base_price, current_price)), 2)


def main_bokeh_app():
    # We are no longer attempting to display inline, focusing on HTML export.
    # output_notebook() # Removed
    # curdoc().add_root(layout) # Removed
    # bokeh_handle = show(layout, notebook_handle=True) # Removed
    # time.sleep(1) # Removed
    print("Bokeh plots will be saved to an HTML file after simulation completes.")


    print("\n--- Starting simulated real-time data processing and visualization ---")
    print("This will run until all data points from dataset.csv are processed.")
    print("You may need to manually stop the cell if it runs indefinitely in a real Pathway setup.")

    # Simulate Pathway streaming by iterating through the DataFrame
    grouped_by_time = df_dataset.groupby('timestamp')
    for timestamp, group in grouped_by_time:
        # Convert group to a list of dictionaries for update_bokeh_data
        records_for_timestep = []
        for _, row in group.iterrows():
            # Call the plain Python functions instead of the Pathway UDFs
            price_model1_val = _calculate_price_model1(row['occupancy'], row['capacity'], row['base_price'])

            demand_value_val = _calculate_demand(row['occupancy'], row['capacity'], row['queue_length'], row['traffic_level'], row['is_special_day'], row['vehicle_type'])
            price_model2_val = _calculate_price_model2(row['base_price'], demand_value_val)

            price_model3_val = _calculate_price_model3(
                price_model2_val,
                row['occupancy'],
                row['capacity'],
                row['base_price'],
                row['competitor_price']
            )

            records_for_timestep.append(
                {
                    'timestamp': row['timestamp'],
                    'parking_space_id': row['parking_space_id'],
                    'latitude': row['latitude'],
                    'longitude': row['longitude'],
                    'capacity': row['capacity'],
                    'occupancy': row['occupancy'],
                    'queue_length': row['queue_length'],
                    'traffic_level': row['traffic_level'],
                    'is_special_day': row['is_special_day'],
                    'vehicle_type': row['vehicle_type'],
                    'competitor_price': row['competitor_price'],
                    'base_price': row['base_price'],
                    'price_model1': price_model1_val,
                    'price_model2': price_model2_val,
                    'price_model3': price_model3_val
                }
            )
        update_bokeh_data(records_for_timestep) # No handle needed here
        time.sleep(0.1) # Simulate real-time delay

    print("\n--- All simulated data processed. Visualization complete. ---")

    # --- Save plots to an HTML file AFTER the simulation completes ---
    html_filename = "dynamic_pricing_plots.html"
    output_file(html_filename)
    save(layout)
    print(f"Plots saved to '{html_filename}'.")
    print(f"Please download '{html_filename}' from the Colab file browser (left sidebar, folder icon) and open it in your web browser to view the plots.")


# Call the main Bokeh app function
main_bokeh_app()


--- CELL 1: Setting up environment and importing libraries ---
Environment setup complete. Libraries imported.

--- CELL 2: Generating synthetic dataset.csv ---
Synthetic dataset 'dataset.csv' generated with 18396 records.
First 5 rows of the generated dataset:
            timestamp  parking_space_id  latitude  longitude  capacity  \
0 2025-01-01 08:00:00               0.0  34.02180 -118.26537     173.0   
1 2025-01-01 08:00:00               1.0  34.06706 -118.22320      57.0   
2 2025-01-01 08:00:00               2.0  34.06686 -118.24052      51.0   
3 2025-01-01 08:00:00               3.0  34.04200 -118.26080     198.0   
4 2025-01-01 08:00:00               4.0  34.02737 -118.29856      68.0   

   occupancy  queue_length  traffic_level  is_special_day vehicle_type  \
0         46             0       0.807971               0          car   
1         12             0       0.917923               0          car   
2         13             0       0.665069               0        truck 